#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/t/ab8726f0cfc84544abbae69a6be88071

## Librerias

In [76]:
import pandas as pd
import numpy as numpy
import urllib.request
from PIL import Image
pd.set_option('display.max.column',None)

## Data

In [77]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
train = pd.read_csv("data/train.csv")

In [78]:
train.shape

(912, 13)

In [79]:
test = pd.read_csv("data/test.csv")

In [80]:
test.shape

(391, 12)

In [81]:
train['dataset'] = 'train'
test['dataset'] = 'test'

In [82]:
df = pd.concat([train, test], ignore_index=True)

In [83]:
df.tail()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,dataset
1298,693,Dell,Alienware 15,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7820HK 2.9GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,3.49kg,NaN,test
1299,1132,Razer,Blade Stealth,Ultrabook,12.5,Touchscreen / 4K Ultra HD 3840x2160,Intel Core i7 6500U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.25kg,NaN,test
1300,49,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4GB,256GB SSD,AMD Radeon R5 M430,Windows 10,2.3kg,NaN,test
1301,1243,Asus,X540SA-RBPDN09 (N3710/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N3710 1.6GHz,4GB,1TB HDD,Intel HD Graphics 405,Windows 10,2.65kg,NaN,test
1302,924,HP,Elitebook Folio,Ultrabook,12.5,4K Ultra HD / Touchscreen 3840x2160,Intel Core M 6Y75 1.2GHz,8GB,240GB SSD,Intel HD Graphics 515,Windows 10,1.09kg,NaN,test


## Data processing

In [84]:
#Nombres columna minúscula
df.columns = df.columns.str.lower()

In [85]:
#Columna ram
df['ram'] = df['ram'].str.rstrip('GB').astype('int')

In [86]:
#Columna weight
df['weight'] = df['weight'].str.rstrip('kg').astype('float')

In [87]:
#Columna company, typename, opsys
cols_to_encode = ['company', 'typename', 'opsys']
df=pd.get_dummies(df, columns=cols_to_encode)
col_bool = df.select_dtypes(include='bool').columns
df[col_bool] = df[col_bool].astype(int)

In [88]:
#Columna product
df['product']= df['product'].str.split(' ', n=1, expand=True)[0]

In [89]:
#Columna screenresolution
df['pixeles'] = df['screenresolution'].str.extract(r'(\d+x\d+)')
df[['pix_ancho', 'pix_largo']] = df['pixeles'].str.split('x', expand=True)
df[['pix_ancho', 'pix_largo']] = df[['pix_ancho', 'pix_largo']].astype(int)

In [90]:
df['touchscreen'] = df['screenresolution'].str.contains('Touchscreen', case=False, na=False)
df['touchscreen']=df['touchscreen'].astype(int)

In [91]:
del df['screenresolution']
del df['pixeles']

In [92]:
#Columna Memory
#capacidad disco duro
df['capacidad_dd'] = df['memory'].str.extract('(\d+(\.\d+)?)\s*(TB|GB)')[0].astype(float)
df.loc[df['memory'].str.contains('TB'), 'capacidad_dd'] *= 1024

#tipo de disco duro
df['tipo_dd'] = df['memory'].str.extract('(SSD|HDD|Hybrid|Flash Storage)')
df_dummies = pd.get_dummies(df['tipo_dd'], prefix='tipo_dd')
df_dummies = df_dummies.astype(int)
df = pd.concat([df, df_dummies], axis=1)
del df['tipo_dd']
del df['memory']

In [93]:
#Columna cpu
df['tipo_cpu'] = df['cpu'].str.split().str[0]
df['tipo_cpu'] = df['tipo_cpu'].apply(lambda x: 1 if 'Intel' in str(x) else 0)
del df['cpu']

In [94]:
#Columna gpu
df['tipo_gpu'] = df['gpu'].str.split().str[0]
df['tipo_gpu'] = df['tipo_gpu'].apply(lambda x: 1 if 'Intel' in str(x) else 0)
del df['gpu']
 

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 51 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   laptop_id                    1303 non-null   int64  
 1   product                      1303 non-null   object 
 2   inches                       1303 non-null   float64
 3   ram                          1303 non-null   int64  
 4   weight                       1303 non-null   float64
 5   price_euros                  912 non-null    float64
 6   dataset                      1303 non-null   object 
 7   company_Acer                 1303 non-null   int64  
 8   company_Apple                1303 non-null   int64  
 9   company_Asus                 1303 non-null   int64  
 10  company_Chuwi                1303 non-null   int64  
 11  company_Dell                 1303 non-null   int64  
 12  company_Fujitsu              1303 non-null   int64  
 13  company_Google    

-----------------------------------------------------------------------------------------------------------------

In [308]:
train = df[df['dataset'] == 'train']
test = df[df['dataset'] == 'test']

In [309]:
train = train.drop(['dataset'], axis=1)
test = test.drop(['dataset'], axis=1)


In [310]:
test.to_csv("data/test1.csv", index=False)

In [311]:
train

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Gpu,OpSys,Weight,Price_euros,capacidad_dd,tipo_dd_Flash Storage,tipo_dd_HDD,tipo_dd_Hybrid,tipo_dd_SSD
0,1104,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,4,AMD Radeon R5,Windows 10,2.40,387.00,500.0,0,1,0,0
1,114,Dell,XPS 13,Ultrabook,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7560U 2.4GHz,8,Intel Iris Plus Graphics 640,Windows 10,1.23,1379.00,256.0,0,0,0,1
2,172,Acer,Aspire A517-51G,Notebook,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8,Nvidia GeForce MX150,Windows 10,3.00,854.00,256.0,0,0,0,1
3,918,HP,Elitebook 820,Netbook,12.5,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8,Intel HD Graphics 620,Windows 10,1.26,1483.00,512.0,0,0,0,1
4,447,Lenovo,IdeaPad 320-15AST,Notebook,17.3,1600x900,AMD A6-Series 9220 2.5GHz,8,AMD Radeon R4,Windows 10,2.80,519.00,1024.0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,996,Lenovo,IdeaPad 320-15ABR,Notebook,15.6,Full HD 1920x1080,AMD A12-Series 9720P 3.6GHz,6,AMD Radeon 530,Windows 10,2.20,549.00,256.0,0,0,0,1
908,731,Dell,Alienware 17,Gaming,17.3,4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,32,Nvidia GeForce GTX 1070,Windows 10,4.36,3659.40,1024.0,0,0,0,1
909,276,Asus,Rog Strix,Gaming,17.3,Full HD 1920x1080,AMD Ryzen 1700 3GHz,16,AMD Radeon RX 580,Windows 10,3.20,1549.00,262144.0,0,0,0,1
910,585,MSI,GT73VR Titan,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7820HK 2.9GHz,16,Nvidia GeForce GTX 1070,Windows 10,4.14,2729.00,524288.0,0,0,0,1


In [312]:
test

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Gpu,OpSys,Weight,Price_euros,capacidad_dd,tipo_dd_Flash Storage,tipo_dd_HDD,tipo_dd_Hybrid,tipo_dd_SSD
912,750,Toshiba,Tecra X40-D-10G,Notebook,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,Windows 10,1.25,NaN,256.0,0,0,0,1
913,726,Mediacom,Smartbook 142,Notebook,14.0,IPS Panel Full HD 1920x1200,Intel Atom Z8350 1.92GHz,4,Intel HD Graphics 400,Windows 10,1.40,NaN,32.0,0,1,0,0
914,633,Mediacom,SmartBook 140,Notebook,14.0,Full HD 1920x1080,Intel Atom x5-Z8350 1.44GHz,2,Intel HD Graphics,Windows 10,1.40,NaN,32.0,1,0,0,0
915,363,Dell,Inspiron 7577,Gaming,15.6,Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8,Nvidia GeForce GTX 1050,Windows 10,2.65,NaN,1024.0,0,1,0,0
916,319,Asus,VivoBook Flip,2 in 1 Convertible,11.6,Touchscreen 1366x768,Intel Celeron Dual Core N3350 1.1GHz,2,Intel HD Graphics 500,Windows 10,1.10,NaN,32.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,693,Dell,Alienware 15,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7820HK 2.9GHz,16,Nvidia GeForce GTX 1070,Windows 10,3.49,NaN,262144.0,0,0,0,1
1299,1132,Razer,Blade Stealth,Ultrabook,12.5,Touchscreen / 4K Ultra HD 3840x2160,Intel Core i7 6500U 2.5GHz,8,Intel HD Graphics 520,Windows 10,1.25,NaN,256.0,0,0,0,1
1300,49,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,AMD Radeon R5 M430,Windows 10,2.30,NaN,256.0,0,0,0,1
1301,1243,Asus,X540SA-RBPDN09 (N3710/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N3710 1.6GHz,4,Intel HD Graphics 405,Windows 10,2.65,NaN,1024.0,0,1,0,0


## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

In [313]:
train = train[['laptop_ID','Inches','Price_euros', 'Ram', 'Weight', 'capacidad_dd','tipo_dd_Flash Storage',	'tipo_dd_HDD','tipo_dd_Hybrid','tipo_dd_SSD']]

In [314]:
train

,laptop_ID,Inches,Price_euros,Ram,Weight,capacidad_dd,tipo_dd_Flash Storage,tipo_dd_HDD,tipo_dd_Hybrid,tipo_dd_SSD
0,1104,15.6,387.00,4,2.40,500.0,0,1,0,0
1,114,13.3,1379.00,8,1.23,256.0,0,0,0,1
2,172,17.3,854.00,8,3.00,256.0,0,0,0,1
3,918,12.5,1483.00,8,1.26,512.0,0,0,0,1
4,447,17.3,519.00,8,2.80,1024.0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
907,996,15.6,549.00,6,2.20,256.0,0,0,0,1
908,731,17.3,3659.40,32,4.36,1024.0,0,0,0,1
909,276,17.3,1549.00,16,3.20,262144.0,0,0,0,1
910,585,17.3,2729.00,16,4.14,524288.0,0,0,0,1


In [315]:
X = train.drop(['Price_euros'], axis=1)
y = train['Price_euros'].copy()
X.shape

(912, 9)

In [316]:
y.shape

(912,)

### 2. Dividir X_train, X_test, y_train, y_test

In [317]:
from sklearn.model_selection import train_test_split

In [318]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [319]:
X_train

,laptop_ID,Inches,Ram,Weight,capacidad_dd,tipo_dd_Flash Storage,tipo_dd_HDD,tipo_dd_Hybrid,tipo_dd_SSD
25,26,15.6,4,2.30,1024.0,0,1,0,0
84,776,15.6,16,2.72,131072.0,0,0,0,1
10,1263,15.6,16,2.50,262144.0,0,0,0,1
342,182,15.6,8,1.90,131072.0,0,0,0,1
890,321,13.3,16,1.29,1024.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
106,394,15.6,8,2.00,256.0,0,0,0,1
270,441,15.6,16,2.50,512.0,0,0,0,1
860,955,17.3,4,2.80,1024.0,0,1,0,0
435,210,15.6,6,2.30,1024.0,0,1,0,0


### 3. Asignar el modelo (vacío) a una variable


In [320]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [321]:
model.fit(X,y)

LinearRegression()

In [322]:
pred = model.predict(X_test)
pred

array([1177.61461639,  351.15518853, 1078.64534809, 1114.5530368 ,
       1958.38312172,  955.5742461 ,  867.34483238, 1263.42704279,
       1086.72435381, 1298.52399031, 1333.93935144,  346.35316915,
        516.82681788, 1315.22543108, 1333.34104895, 1820.11579169,
       1976.37370524,  964.80509263, 1116.66736465,  329.57900251,
       1060.72110628,  553.68736462, 1098.76586888, 1165.1834071 ,
       1046.64026967,  402.24205986,  955.50329326, 1846.14360853,
       1082.8604815 ,  511.72835328, 1248.47212565, 1176.76509026,
        970.74052859, 1170.10187572, 1313.36982684,  386.31144178,
        950.36257312,  589.46915892, 1280.14284224,  904.73233298,
       1289.57727569, 1174.81284814, 1212.81213903, 1881.56742337,
        977.19277928,  423.85604418, 1831.57125211,  502.09035315,
       1135.85483036,  795.06320847, 2168.39860312, 1210.93313298,
        385.18166597, 1839.01877792,  975.74485913, 1279.93809002,
       2132.87272618, 1076.99905519, 1936.93766869, 1913.36440

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

In [323]:
from sklearn.metrics import mean_squared_error
import numpy as np

print('RMSE :',np.sqrt(mean_squared_error(y_test,pred)))

RMSE : 447.67569262745235


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

In [334]:
X_pred = pd.read_csv("data/test1.csv")
X_pred.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Gpu,OpSys,Weight,Price_euros,capacidad_dd,tipo_dd_Flash Storage,tipo_dd_HDD,tipo_dd_Hybrid,tipo_dd_SSD
0,750,Toshiba,Tecra X40-D-10G,Notebook,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,Windows 10,1.25,NaN,256.0,0,0,0,1
1,726,Mediacom,Smartbook 142,Notebook,14.0,IPS Panel Full HD 1920x1200,Intel Atom Z8350 1.92GHz,4,Intel HD Graphics 400,Windows 10,1.40,NaN,32.0,0,1,0,0
2,633,Mediacom,SmartBook 140,Notebook,14.0,Full HD 1920x1080,Intel Atom x5-Z8350 1.44GHz,2,Intel HD Graphics,Windows 10,1.40,NaN,32.0,1,0,0,0
3,363,Dell,Inspiron 7577,Gaming,15.6,Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8,Nvidia GeForce GTX 1050,Windows 10,2.65,NaN,1024.0,0,1,0,0
4,319,Asus,VivoBook Flip,2 in 1 Convertible,11.6,Touchscreen 1366x768,Intel Celeron Dual Core N3350 1.1GHz,2,Intel HD Graphics 500,Windows 10,1.10,NaN,32.0,1,0,0,0


In [337]:
X_pred=X_pred[['laptop_ID','Inches','Ram', 'Weight', 'capacidad_dd','tipo_dd_Flash Storage',	'tipo_dd_HDD','tipo_dd_Hybrid','tipo_dd_SSD']]

In [338]:
X_pred

,laptop_ID,Inches,Ram,Weight,capacidad_dd,tipo_dd_Flash Storage,tipo_dd_HDD,tipo_dd_Hybrid,tipo_dd_SSD
0,750,14.0,8,1.25,256.0,0,0,0,1
1,726,14.0,4,1.40,32.0,0,1,0,0
2,633,14.0,2,1.40,32.0,1,0,0,0
3,363,15.6,8,2.65,1024.0,0,1,0,0
4,319,11.6,2,1.10,32.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
386,693,15.6,16,3.49,262144.0,0,0,0,1
387,1132,12.5,8,1.25,256.0,0,0,0,1
388,49,15.6,4,2.30,256.0,0,0,0,1
389,1243,15.6,4,2.65,1024.0,0,1,0,0


**¿Por qué da error?** 

In [339]:
predictions_submit = model.predict(X_pred)
predictions_submit

array([1250.36079706,  538.71250239,  287.39902767,  817.56619279,
        343.57068802,  700.31727417,  286.644572  , 1976.53278028,
       1896.48855046, 1251.67529567, 1060.97059431, 1172.47467396,
       1163.07112683, 1111.13700152,  696.54829987, 1193.14151641,
       1173.51073762, 2012.76377686,  450.63540994,  905.01151595,
       1329.7700706 , 1039.51934359,  587.84498858, 1277.36903492,
        835.62611093, 1243.35253303,  779.56192749,  722.3230575 ,
        730.78128774, 1210.92900595, 1070.99005221, 1136.83380273,
       1066.25202065, 1257.31346981, 1235.23960101, 1232.06712462,
       1187.7054851 , 1364.82176309,  909.53889397, 2034.53746578,
       1939.93032711, 1079.73297829,  458.55415325, 1160.09140431,
       1076.84585897, 1944.96924224,  762.15619963, 1842.77213382,
       1186.55678907, 1982.03387421,  556.83726855,  412.54517037,
        785.03799137, 1266.01034574,  706.12245616,  847.91835465,
       1137.47224872,  652.22828046, 1336.30615228, 1287.58507

IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [ ]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [340]:
predictions_submit = model.predict(X_pred)
predictions_submit

array([1250.36079706,  538.71250239,  287.39902767,  817.56619279,
        343.57068802,  700.31727417,  286.644572  , 1976.53278028,
       1896.48855046, 1251.67529567, 1060.97059431, 1172.47467396,
       1163.07112683, 1111.13700152,  696.54829987, 1193.14151641,
       1173.51073762, 2012.76377686,  450.63540994,  905.01151595,
       1329.7700706 , 1039.51934359,  587.84498858, 1277.36903492,
        835.62611093, 1243.35253303,  779.56192749,  722.3230575 ,
        730.78128774, 1210.92900595, 1070.99005221, 1136.83380273,
       1066.25202065, 1257.31346981, 1235.23960101, 1232.06712462,
       1187.7054851 , 1364.82176309,  909.53889397, 2034.53746578,
       1939.93032711, 1079.73297829,  458.55415325, 1160.09140431,
       1076.84585897, 1944.96924224,  762.15619963, 1842.77213382,
       1186.55678907, 1982.03387421,  556.83726855,  412.54517037,
        785.03799137, 1266.01034574,  706.12245616,  847.91835465,
       1137.47224872,  652.22828046, 1336.30615228, 1287.58507

In [ ]:
print('RMSE :',np.sqrt(mean_squared_error(test,pred)))

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [341]:
sample = pd.read_csv("data/sample_submission.csv")

In [330]:
sample.head()

,laptop_ID,Price_euros
0,750,500
1,726,500
2,633,500
3,363,650
4,319,650


In [342]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [343]:
submission = pd.DataFrame({"laptop_ID": sample['laptop_ID'], "Price_euros": predictions_submit})

In [344]:
submission.head()

,laptop_ID,Price_euros
0,750,1250.360797
1,726,538.712502
2,633,287.399028
3,363,817.566193
4,319,343.570688


In [28]:
submission.shape

(391, 2)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [345]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [346]:
chequeator(submission)

You're ready to submit!
